In [1]:
import numpy as np

def alpha_to_mass(alpha):
    """
    将 Dirichlet 参数 alpha 转换为 DS Mass 和 Uncertainty
    Input: alpha (..., K)
    Output: mass (..., K), uncertainty (..., 1)
    """
    S = np.sum(alpha, axis=-1, keepdims=True) # S = sum(alpha)
    K = alpha.shape[-1]
    
    # Mass k = (alpha_k - 1) / S
    mass = (alpha - 1.0) / S
    
    # Uncertainty = K / S
    uncertainty = K / S
    
    # 裁剪防止数值误差导致负数 (理论上 alpha >= 1)
    mass = np.maximum(mass, 0.0)
    
    return mass, uncertainty

def dempster_fusion(m1, m2, u1, u2):
    """
    DS 组合规则融合两个 Mass 分布
    m1, m2: (K,) 或 (N, K) 的 Mass 向量
    u1, u2: (1,) 或 (N, 1) 的 Uncertainty 标量
    """
    # 1. 计算冲突因子 C (Conflict)
    # C = sum_{i!=j} m1[i] * m2[j]
    # 可以通过外积计算所有对的乘积，然后减去对角线(i==j)的部分
    
    # 为了支持批量计算 (N, K)，我们使用 einsum 或者 广播
    # 这里展示单点逻辑的向量化版本
    
    # 计算所有两两乘积矩阵 (N, K, K)
    # product_matrix[n, i, j] = m1[n, i] * m2[n, j]
    product_matrix = np.einsum('...i,...j->...ij', m1, m2)
    
    # 对角线部分 (i==j):用于共识
    diagonal = np.diagonal(product_matrix, axis1=-2, axis2=-1) # (..., K)
    consensus = diagonal
    
    # 冲突 C = 所有元素和 - 对角线元素和
    C = np.sum(product_matrix, axis=(-2, -1)) - np.sum(consensus, axis=-1)
    
    # 2. 归一化因子
    norm_factor = 1.0 - C
    # 防止除以0
    norm_factor = np.maximum(norm_factor, 1e-8)
    
    # 3. 融合公式
    # m_new[i] = (m1[i]*m2[i] + m1[i]*u2 + m2[i]*u1) / (1-C)
    
    term1 = consensus           # m1[i] * m2[i]
    term2 = m1 * u2             # m1[i] * u2 (广播)
    term3 = m2 * u1             # m2[i] * u1 (广播)
    
    m_new = (term1 + term2 + term3) / norm_factor[..., None]
    
    # 计算新的不确定性 (理论上 u_new = 1 - sum(m_new))
    # 或者用公式 u_new = (u1 * u2) / (1-C)
    u_new = (u1 * u2) / norm_factor[..., None]
    
    return m_new, u_new

# === 使用示例 ===
# 假设从 .npy 读取了 alpha
alpha_obs = np.array([2.0, 1.2, 1.5]) # 假设3类
current_map_mass = np.array([0.0, 0.0, 0.0]) # 初始全0
current_map_unc = 1.0

# 1. 转换
obs_mass, obs_unc = alpha_to_mass(alpha_obs)

# 2. 融合
# 如果是第一次 (num=0)，直接赋值
current_map_mass = obs_mass
current_map_unc = obs_unc

# 如果是第二次观测
alpha_obs_2 = np.array([5.0, 1.1, 1.1])
obs_mass_2, obs_unc_2 = alpha_to_mass(alpha_obs_2)

fused_mass, fused_unc = dempster_fusion(current_map_mass, obs_mass_2, current_map_unc, obs_unc_2)

print("Fused Mass:", fused_mass)
print("Fused Unc:", fused_unc)

Fused Mass: [0.61748456 0.02989925 0.0601235 ]
Fused Unc: [0.29249269]


In [2]:
import numpy as np

# 1. 基础配置
VOXEL_RESOLUTION = 0.2  # 分辨率，单位：米 (原项目典型值)
NUM_CLASSES = 9         # 类别数

# 2. 地图容器 (稀疏体素网格)
# Key: (x_idx, y_idx, z_idx)
# Value: { 'mass': np.array, 'uncertainty': float }
voxel_map = {} 

def get_voxel_index(point, resolution):
    """将物理坐标转换为体素索引"""
    return tuple(np.floor(point / resolution).astype(int))

def update_voxel_map(voxel_map, points, point_alphas, resolution):
    """
    更新体素地图的主函数
    points: (N, 3) 点云坐标
    point_alphas: (N, K) 每个点的 Dirichlet 参数 (from .npy)
    """
    # 预先计算所有点的体素索引 (向量化加速)
    voxel_indices = np.floor(points / resolution).astype(int)
    
    # 遍历每个点进行融合
    # 注意：为了效率，实际工程中通常会先对 indices 进行 argsort 或 unique，
    # 将属于同一个体素的点先在这一帧内合并，再与全局地图融合。
    # 这里为了演示逻辑，使用简单的逐点循环。
    
    for i in range(len(points)):
        idx = tuple(voxel_indices[i])
        
        # 1. 获取当前观测的 Mass 和 Uncertainty
        # (假设您已经实现了上一步的 alpha_to_mass 函数)
        obs_alpha = point_alphas[i]
        obs_mass, obs_unc = alpha_to_mass(obs_alpha) # 需定义此函数
        
        if idx not in voxel_map:
            # --- 初始化新体素 ---
            voxel_map[idx] = {
                'mass': obs_mass,
                'uncertainty': obs_unc,
                'count': 1
            }
        else:
            # --- 融合旧体素 ---
            curr_voxel = voxel_map[idx]
            old_mass = curr_voxel['mass']
            old_unc = curr_voxel['uncertainty']
            
            # 调用 Dempster 组合规则 (需定义此函数)
            new_mass, new_unc = dempster_fusion(old_mass, obs_mass, old_unc, obs_unc)
            
            # 更新状态
            curr_voxel['mass'] = new_mass
            curr_voxel['uncertainty'] = new_unc
            curr_voxel['count'] += 1

    return voxel_map

# 3. 将地图转换为点云用于可视化
def map_to_pointcloud(voxel_map, resolution):
    points = []
    colors = [] # 或 semantics
    uncertainties = []
    
    for idx, data in voxel_map.items():
        # 恢复物理坐标 (取体素中心)
        center_point = (np.array(idx) * resolution) + (resolution / 2.0)
        
        # 获取最大概率类别
        mass = data['mass']
        label = np.argmax(mass)
        
        points.append(center_point)
        colors.append(label) # 这里存 label，后续用 get_rellis_colors 转颜色
        uncertainties.append(data['uncertainty'])
        
    return np.array(points), np.array(colors), np.array(uncertainties)